In [1]:
print('kernel_on')

kernel_on


# Exploratory Analysis on DialogRE:
- **Objective**: Analyze dialogue relations data from JSON files & estimate data synthesization costs
- **Process**: Convert data to DataFrame, flatten Relations column, and extract `relation_type`
- **Analysis**: Study distribution of `relation_type`, with a special focus on excluding `no_relation`

In [6]:
from src.paths import LOCAL_PROCESSED_DATA_PATH
from src.statistics import get_counts_and_percentages

from pathlib import Path
import pandas as pd
import json
import glob
import math

# Get a list of all json files in the directory, excluding 'dev'
for folder in ("with-no-relation", "ternary", "binary"):
    files = [Path(f) for f in glob.glob(str(LOCAL_PROCESSED_DATA_PATH / f"dialog-re-{folder}/*.json"))]
    
    # Create an empty DataFrame
    df1 = pd.DataFrame(columns=["Dialogue", "Relations", "Origin"])
    
    # Loop over all json files in the directory
    for file_name in files:
        with open(file_name, 'r') as file:
            data = json.load(file)
    
            # Convert the data to a DataFrame
            df_temp = pd.DataFrame(data, columns=["Dialogue", "Relations"])
    
            # Add a new column to this DataFrame for the origin
            df_temp["Origin"] = file_name.stem  # This will get just the file name without the extension
    
            # Append the temporary DataFrame to the main DataFrame
            df1 = pd.concat([df1, df_temp], ignore_index=True)
    
    # flatten Relations column
    relations = df1['Relations'].apply(pd.Series).stack().reset_index(drop=True)
    
    # create new dataframe
    df2 = pd.DataFrame({'Relations': relations})
    
    # extract 'relation_type' from dictionary and add it as a new column in df2
    df2['relation_type'] = df2['Relations'].apply(lambda x: x['r'][0])
    
    # drop the 'Relations' column
    df2 = df2.drop('Relations', axis=1)
    
    # Now, we can count the amount of each 'relation_type'
    relation_stats = get_counts_and_percentages(df2, ['relation_type'])

    print(f"dialog-re-{folder}/*.json")
    display(relation_stats)

dialog-re-with-no-relation/*.json


,Counts,%
relation_type,,
no_relation,16489,62.8
per:alternate_names,2136,8.1
unanswerable,2099,8.0
per:girl/boyfriend,736,2.8
per:positive_impression,657,2.5
per:friends,648,2.5
per:title,414,1.6
per:spouse,316,1.2
per:siblings,303,1.2


dialog-re-ternary/*.json


,Counts,%
relation_type,,
no_relation,16489,62.8
with_relation,7650,29.2
unanswerable,2099,8.0


dialog-re-binary/*.json


,Counts,%
relation_type,,
no_relation_unanswerable,18588,70.8
with_relation,7650,29.2
